In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("/Users/macbookair/Downloads/transactional-sample.txt")

In [5]:
df.head()

,transaction_id,merchant_id,user_id,card_number,transaction_date,transaction_amount,device_id,has_cbk
0,21320398,29744,97051,434505******9116,2019-12-01 23:16:32.812632,374.56,285475.0,False
1,21320399,92895,2708,444456******4210,2019-12-01 22:45:37.873639,734.87,497105.0,True
2,21320400,47759,14777,425850******7024,2019-12-01 22:22:43.021495,760.36,NaN,False
3,21320401,68657,69758,464296******3991,2019-12-01 21:59:19.797129,2556.13,NaN,True
4,21320402,54075,64367,650487******6116,2019-12-01 21:30:53.347051,55.36,860232.0,False


In [ ]:
## convert transaction_date to datetime

df['transaction_date'] = pd.to_datetime(df['transaction_date'])

In [6]:
### check chargeback rate per user, card, device, merchant
print("Top users with most chargebacks:")
print(df[df['has_cbk'] == True].groupby('user_id')['transaction_id'].count().sort_values(ascending=False).head(10))

Top users with most chargebacks:
user_id
11750    25
91637    19
79054    15
96025    13
78262    12
75710    10
7725      7
17929     6
21768     6
28218     5
Name: transaction_id, dtype: int64


In [7]:
print("\nTop devices with most chargebacks:")
print(df[df['has_cbk'] == True].groupby('device_id')['transaction_id'].count().sort_values(ascending=False).head(10))


Top devices with most chargebacks:
device_id
563499.0    19
342890.0    15
101848.0    15
438940.0    13
547440.0    12
960729.0     6
542535.0     6
357277.0     6
308950.0     5
597308.0     5
Name: transaction_id, dtype: int64


In [8]:
print("\nTop cards with most chargebacks:")
print(df[df['has_cbk'] == True].groupby('card_number')['transaction_id'].count().sort_values(ascending=False).head(10))



Top cards with most chargebacks:
card_number
554482******7640    10
530034******3859     6
651653******2256     5
530034******8258     4
498406******7104     4
552289******8870     4
459080******2870     4
441030******2146     4
552693******2902     3
552640******7443     3
Name: transaction_id, dtype: int64


In [9]:
### Check for repeated transactions
df_sorted = df.sort_values(['card_number', 'transaction_date'])
df_sorted['prev_time'] = df_sorted.groupby('card_number')['transaction_date'].shift(1)
df_sorted['diff_minutes'] = (df_sorted['transaction_date'] - df_sorted['prev_time']).dt.total_seconds() / 60

In [10]:
rapid_transactions = df_sorted[(df_sorted['diff_minutes'] < 10) & (df_sorted['diff_minutes'] >= 0)]
print("\nRapid repeat transactions with same card:")
print(rapid_transactions[['transaction_id', 'card_number', 'transaction_date', 'diff_minutes']].head(10))



Rapid repeat transactions with same card:
      transaction_id       card_number           transaction_date  \
1714        21322112  230744******4802 2019-11-22 22:41:36.534791   
2755        21323153  406168******4222 2019-11-11 23:20:43.945517   
758         21321156  406669******1721 2019-11-28 23:59:44.770666   
756         21321154  406669******1721 2019-11-29 00:01:41.332782   
629         21321027  410863******7755 2019-11-29 15:35:24.506721   
628         21321026  410863******7755 2019-11-29 15:36:16.283879   
138         21320536  415274******8162 2019-12-01 02:16:11.464498   
964         21321362  415944******1540 2019-11-28 15:41:40.107253   
1030        21321428  415944******7186 2019-11-28 02:51:57.935225   
481         21320879  421960******7769 2019-11-29 21:07:27.923685   

      diff_minutes  
1714      9.749440  
2755      5.791082  
758       5.307498  
756       1.942702  
629       1.432361  
628       0.862953  
138       7.594835  
964       0.955325  
1030    

In [ ]:
# Devices used by multiple users (potential synthetic ID fraud)
device_user_counts = df.groupby('device_id')['user_id'].nunique().sort_values(ascending=False)
print("\nDevices used by most unique users:")
print(device_user_counts.head(10))


Devices used by most unique users:
device_id
999843.0    1
2.0         1
4.0         1
486.0       1
2359.0      1
2630.0      1
3044.0      1
3290.0      1
4076.0      1
4571.0      1
Name: user_id, dtype: int64


In [ ]:
# Devices used by multiple users (potential synthetic ID fraud)
device_user_counts = df.groupby('device_id')['user_id'].nunique().sort_values(ascending=False)
print("\nDevices used by most unique users:")
print(device_user_counts.head(10))


Devices used by most unique users:
device_id
999843.0    1
2.0         1
4.0         1
486.0       1
2359.0      1
2630.0      1
3044.0      1
3290.0      1
4076.0      1
4571.0      1
Name: user_id, dtype: int64


In [ ]:
# Merchants with most chargebacks
merchant_cbk = df[df['has_cbk'] == True].groupby('merchant_id')['transaction_id'].count().sort_values(ascending=False)
print("\nMerchants with most chargebacks:")
print(merchant_cbk.head(10))


Merchants with most chargebacks:
merchant_id
17275    22
4705     19
1308     15
53041    14
77130    13
44927    11
91972    11
73271    10
29214     9
55854     9
Name: transaction_id, dtype: int64
